In [ ]:
import os
import sys
from cv2 import imread
from os import popen
from nydataloader import EvalDataset
from evaluator import Eval_thread
import pickle
import torch
import random
# 设置使用哪张卡
torch.cuda.set_device(1)

# 使用说明

1. 规范目录结构
   DUT-OMRON、DUTS-7-8等对应下面代码中的dataset_names
   gt_dir和pred_dir都要有dataset_names说明的这些文件夹
   pred_dir对应的是预测结果文件夹的根目录，以里面的一个子文件夹train-mode-semi-4_result为例
   完整结构如下图所示
```
|-------pred_dir
    ├── train-mode-semi-4_result
    │   ├── DUT-OMRON
    │   ├── DUTS-7-8
    │   ├── DUTS-TE
    │   ├── ECSSD
    │   ├── HKU-IS
    │   ├── PASCAL-S
    │   └── SOD
    
    ├── train-mode-semi-5_result
    │   ├── DUT-OMRON
    │   ├── DUTS-7-8
    │   ├── DUTS-TE
    │   ├── ECSSD
    │   ├── HKU-IS
    │   ├── PASCAL-S
    │   └── SOD
    
    ├── ........
    │   ├── DUT-OMRON
    │   ├── DUTS-7-8
    │   ├── DUTS-TE
    │   ├── ECSSD
    │   ├── HKU-IS
    │   ├── PASCAL-S
    │   └── SOD
```
   gt_dir目录结构如下图所示，这里建议使用软链接链接过去
```
├── gt
│   ├── DUT-OMRON -> /home/ai-server/disk2/thb/dataset/DUT-OMRON/GT/
│   ├── DUTS-7-8 -> /home/ai-server/disk2/thb/dataset/DUTS/DUTS-TE/TEST-7-8/GT
│   ├── DUTS-TE -> /home/ai-server/disk2/thb/dataset/DUTS/DUTS-TE/DUTS-TE-Mask/
│   ├── ECSSD -> /home/ai-server/disk2/thb/dataset/ECSSD/GT/
│   ├── HKU-IS -> /home/ai-server/disk2/thb/dataset/HKU-IS/GT/
│   ├── PASCAL-S -> /home/ai-server/disk2/thb/dataset/PASCAL-S/GT/
│   └── SOD -> /home/ai-server/disk2/thb/dataset/SOD/GT/
```

2. 多个模型对应多个预测结果
    model_names即对应预测结果文件夹，这是一个可迭代的元素
    
3. salmetric_cmd_path 对应C++版本的测试程序



In [ ]:
# dataset_names = ('DUTS-7-8','DUT-OMRON','DUTS-TE','ECSSD','HKU-IS','PASCAL-S','SOD')
dataset_names = ('DUT-OMRON','DUTS-TE','ECSSD','HKU-IS','PASCAL-S','SOD')
# dataset_names = ('DUTS-TE',)
gt_dir = '/home/ai-server/disk2/thb/pycode/Evaluation-SOD/data-2/gt'
# model_name is in the pred_dir  
pred_dir = '/home/thb/MyAICode/pycode/AdvSaliency/'

salmetric_cmd_path = "/home/thb/MyAICode/dataset/DUTS/DUTS-TE/salmetric"
lst_file_path = './tmp01-%d.lst' % random.randint()
# model_names = ['d-learning-rate_%s_result' % s for s in ("0.0009","0.001",)]
# model_names = ['learning-rate_%s_result' % i for i in ('0.0001','0.0006','0.0009') ]
# model_names = ['train-without-D_result','train-without-semi_result','train-without-adv_result','train-with-dmap_loss_result']
model_names = ['train-without-D-%d_result' % i for i in (1000,2000)]
res_dict = dict()

In [ ]:
gt_datasets = [ os.path.join(gt_dir,dataset_name) for dataset_name in dataset_names ]
gt_datasets

In [ ]:
pred_model_dirs = [os.path.join(pred_dir,model_name) for model_name in model_names] 
pred_model_dirs

In [ ]:
def check_dir_exists(gt_dir,pred_dir):
    # get max-s
    image_path_list = list()
    label_path_list = list()
    
    for image_name in os.listdir(gt_dir):
        label_path = os.path.join(gt_dir, image_name)
        image_path = os.path.join(pred_dir, image_name)
        if os.path.exists(image_path):
            image_path_list.append(image_path)
            label_path_list.append(label_path)
        else:
            basename = os.path.splitext(image_name)[0]
            image_path = os.path.join(pred_dir, basename+'.jpg')
#             print(image_path)
            if os.path.exists(image_path):
                image_path_list.append(image_path)
                label_path_list.append(label_path)  
    print(len(image_path_list))
    if len(image_path_list) == 0:
        print(pred_dir)            



In [ ]:
for dataset_name,gt_dataset_dir in zip(dataset_names,gt_datasets):
    for model_name,pred_model_dir in zip(model_names,pred_model_dirs):
        pred_dataset_dir = os.path.join(pred_model_dir ,dataset_name)
        check_dir_exists(gt_dataset_dir, pred_dataset_dir)


In [ ]:

def get_mae_and_max_f_and_s(gt_dir,pred_dir):
    # get max-s
    loader = EvalDataset(pred_dir, gt_dir)
    thread = Eval_thread(loader, True)
    s_dict = thread.run()
    print(s_dict)
    
    image_path_list = list()
    label_path_list = list()
    
    for image_name in os.listdir(gt_dir):
        label_path = os.path.join(gt_dir, image_name)
        image_path = os.path.join(pred_dir, image_name)
        if os.path.exists(image_path):
            image_path_list.append(image_path)
            label_path_list.append(label_path)
        else:
            basename = os.path.splitext(image_name)[0]
            image_path = os.path.join(pred_dir, basename+'.jpg')
#             print(image_path)
            if os.path.exists(image_path):
                image_path_list.append(image_path)
                label_path_list.append(label_path)  
                
    with open(lst_file_path,'w',encoding="utf-8") as fp:
        for image_path,label_path in zip(image_path_list,label_path_list):
            img_obj = imread(image_path)
            gt_obj = imread(label_path)
            if img_obj is not None and gt_obj is not None and \
            gt_obj.shape == img_obj.shape:
                fp.write(image_path + ' ' + label_path +'\n')
    cmd_str = salmetric_cmd_path + ' ' + lst_file_path + ' 10'
    rd_lines = popen(cmd_str).readlines()
    rd_lines = rd_lines[-4:]
    eval_result_dict = dict()
    eval_result_dict['Max F-measre'] = 0.0
    eval_result_dict['MAE'] = 0.0
    eval_result_dict.update(s_dict)
    if len(rd_lines) == 4:
        rd_lines = rd_lines[0:len(rd_lines):len(rd_lines)-1]
        if len(rd_lines) == 2:
            for line in rd_lines:
                w_data = line.strip()
                w_data = w_data.split(':')
                if len(w_data) == 2:
                     eval_result_dict[w_data[0].strip()] = float(w_data[1].strip())
    return eval_result_dict

In [ ]:
for dataset_name,gt_dataset_dir in zip(dataset_names,gt_datasets):
    for model_name,pred_model_dir in zip(model_names,pred_model_dirs):
        print(model_name,dataset_name,)
        pred_dataset_dir = os.path.join(pred_model_dir ,dataset_name)
        ret = get_mae_and_max_f_and_s(gt_dataset_dir, pred_dataset_dir)
        print(ret)
        try:
            res_dict[model_name][dataset_name] = ret
        except:
            res_dict[model_name] = dict()
            res_dict[model_name][dataset_name] = ret

In [ ]:
with open('spl2.pkl','wb') as f:
    pickle.dump(res_dict,f)
# import pickle
# with open('spl2.pkl','rb') as f:
#     res_dict = pickle.load(f)

In [ ]:
# id_list = ['PoolNet','BASNet','BRN','MLMSNet','PAGE-Net','PAGRN']
id_list = model_names
dataset_list = ['ECSSD','PASCAL-S','DUT-OMRON','HKU-IS','SOD','DUTS-TE']
# dataset_list = ['DUTS-7-8','ECSSD','PASCAL-S','DUT-OMRON','HKU-IS','SOD','DUTS-TE']
# dataset_list = ['DUTS-TE',]
# dataset_list = dataset_names
metric_list = ['Max F-measre','MAE','max-S']

In [ ]:
dataframe = dict()
dataframe['ID'] = id_list
for dataset in dataset_list:
    for metric in metric_list:
        dataframe[dataset+' '+metric] = [res_dict[id_name][dataset][metric] for id_name in id_list]

In [ ]:
import pandas as pd

In [ ]:
new_dataframe = pd.DataFrame(dataframe)
print(new_dataframe)

In [ ]:
new_dataframe.to_excel('./1000-2000.xlsx')

In [ ]:
# 删除临时文件
os.unlink(lst_file_path)